Я использовала список тегов из https://universaldependencies.org/u/pos/ , но CCONJ и SCONJ заменила просто CONJ. Я посмотрела на список выбранных POS теггеров (spacy, mystem и stanza) и такой списог тегов был самым удобным, потому что в нем не было такого, что двумя разными тегами обозначено в нем, но одним тегом — в каком-то из теггеров (то есть, на примере с теми же CCONJ и SCONJ, там, где они были, я могла их просто вместе заменить на CONJ, а там, где и так был только CONJ, оставался он)

Корпус у меня был составлен из разных предложений с омонимией, где сложно определить часть речи

In [2]:
import re
import spacy
nlp = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])
tokens = []
pos = []
newfile = []
with open('textforpostag.txt',  encoding='utf-8') as f:
    for line in f:
        line = re.findall(r'\w+', line)
        line = " ".join(line)
        doc = nlp(line)
        for token in doc:
            tokens.append(token.text)
            pos.append(token.pos_)

for word in pos:
    if word == 'CCONJ' or word == 'SCONJ':
        newfile.append('CONJ')
    else:
        newfile.append(word)

   

In [3]:
import stanza
ppln = stanza.Pipeline('ru', processors='tokenize,pos')


2022-10-11 23:14:33 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-11 23:14:35 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |

2022-10-11 23:14:35 INFO: Use device: cpu
2022-10-11 23:14:35 INFO: Loading: tokenize
2022-10-11 23:14:35 INFO: Loading: pos
2022-10-11 23:14:36 INFO: Done loading processors!


In [31]:
import re

files = []
newfiles = []
with open('textforpostag.txt',  encoding='utf-8') as f:
    for line in f:
        line = re.findall(r'\w+', line)
        line = " ".join(line)
        doc = ppln(line)
        for snt in doc.sentences:
            for word in snt.words:
                file = f'{word.upos}'
                files.append(file)

for word in files:
    if word == 'CCONJ' or word == 'SCONJ':
        newfiles.append('CONJ')
    elif word == 'AUX':
        newfiles.append('VERB')
    else:
        newfiles.append(word)
print(newfiles)      

['ADP', 'PRON', 'VERB', 'NOUN', 'ADV', 'VERB', 'ADJ', 'PRON', 'PART', 'VERB', 'VERB', 'ADP', 'NOUN', 'PRON', 'VERB', 'ADV', 'VERB', 'ADP', 'NOUN', 'PART', 'VERB', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'VERB', 'CONJ', 'ADJ', 'NOUN', 'PART', 'VERB', 'PART', 'PRON', 'PART', 'VERB', 'CONJ', 'VERB', 'VERB', 'PART', 'VERB', 'NOUN', 'PRON', 'ADV', 'VERB', 'ADP', 'NOUN', 'ADJ', 'ADP', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'DET', 'NOUN', 'ADJ', 'PRON', 'PART', 'VERB', 'ADV', 'NOUN', 'VERB', 'ADV', 'CONJ', 'PRON', 'VERB', 'ADJ', 'ADV', 'PART', 'VERB', 'ADP', 'DET', 'NOUN', 'ADV', 'NOUN', 'VERB', 'VERB', 'ADV', 'VERB', 'NOUN', 'VERB', 'VERB', 'PRON', 'DET', 'NOUN', 'ADP', 'NOUN', 'PRON', 'ADJ', 'PRON', 'PART', 'VERB', 'PART', 'ADP', 'PRON', 'PART', 'VERB', 'PRON', 'ADJ', 'VERB', 'ADV', 'CONJ', 'ADV', 'NOUN', 'VERB', 'CONJ', 'PRON', 'VERB', 'VERB', 'NOUN', 'NOUN', 'VERB', 'VERB', 'NOUN', 'VERB', 'NOUN', 'VERB', 'NOUN', 'VERB', 'ADP', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'ADV', 'ADP', '

In [19]:
from pymystem3 import Mystem
m = Mystem()

tokens1 = []
pos1 = []
newfile1 = []
with open('textforpostag.txt',  encoding='utf-8') as f:
    for line in f:
        line = re.findall(r'\w+', line)
        line = " ".join(line)
        ana = m.analyze(line)
        for word in ana:
            if 'analysis' in word:
                gr = word['analysis'][0]['gr']
                pos = gr.split('=')[0].split(',')[0]
                tokens1.append(word['text']) 
                pos1.append(pos)

for word in pos1:
    if word == 'PR':
        newfile1.append('ADP')
    elif word == 'S':
        newfile1.append('NOUN')
    elif word == 'A':
        newfile1.append('ADJ')
    elif word == 'V':
        newfile1.append('VERB')
    elif word == 'SPRO' or word == 'APRO' or word == 'ADVPRO':
        newfile1.append('PRON')
    else:
        newfile1.append(word)

In [14]:
with open('corpuspos.txt',  'r') as f:
    linepred = f.readline().split()

In [27]:
from sklearn.metrics import accuracy_score, f1_score
accuracy_score(newfile, linepred)

0.8578431372549019

In [33]:
accuracy_score(newfile1, linepred)

0.8725490196078431

In [34]:
accuracy_score(newfiles, linepred)

0.8676470588235294

Самый лучший теггер — mystem